In [1]:
import os
import shutil

import tensorflow as tf

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

#import the data set data/hatespeech.csv
data_path = os.path.join(os.getcwd(), 'data/hatespeech.csv')
"""Loads the data set."""
# Load the data set
dataset = tf.data.experimental.CsvDataset(
    data_path, [tf.float32, tf.string], header=True)
# Split into train/eval
train_size = int(0.8 * dataset.cardinality())
train_dataset = dataset.skip(train_size)
eval_dataset = dataset.skip(train_size)
# Shuffle, repeat, and batch the examples.
train_dataset = train_dataset.shuffle(1000).repeat().batch(32)
eval_dataset = eval_dataset.shuffle(1000).repeat().batch(32)
print(train_dataset, eval_dataset)


2022-02-02 16:24:02.850617: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-02-02 16:24:02.850862: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



TypeError: Cannot convert 0.8 to EagerTensor of dtype int64